<a href="https://colab.research.google.com/github/mk7890/Resume-Parsing-System/blob/main/ResumeParsingSystem_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# jobs_dataset_with_features : https://drive.google.com/file/d/1vAaazotBSAS6tAZ17l6d1x7rCjzRp5te/view?usp=sharing
# clean_resume_data : https://drive.google.com/file/d/1CytY9v0F--lPARegLYEXjnXXzR-_gC-T/view?usp=sharing
# model link : https://drive.google.com/file/d/1LW6366Q0CS2ZYvKiPrYezPGxbl7MCsnN/view?usp=sharing

In [4]:
import gdown

# Replace 'FILE_ID' with the actual file ID from your link
file_id = "1CytY9v0F--lPARegLYEXjnXXzR-_gC-T"
output_filename = "clean_resume_data.csv"  # Change to desired output filename

# Construct the download URL
url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(url, output_filename, quiet=False)

print(f"File downloaded as: {output_filename}")


Downloading...
From: https://drive.google.com/uc?id=1CytY9v0F--lPARegLYEXjnXXzR-_gC-T
To: /content/clean_resume_data.csv
100%|██████████| 12.2M/12.2M [00:00<00:00, 32.8MB/s]

File downloaded as: clean_resume_data.csv


In [5]:
import gdown

# Replace 'FILE_ID' with the actual file ID from your link
file_id2 = "1vAaazotBSAS6tAZ17l6d1x7rCjzRp5te"
output_filename2 = "resume_dataset_with_features.csv"  # Change to desired output filename

# Construct the download URL
url = f"https://drive.google.com/uc?id={file_id2}"

# Download the file
gdown.download(url, output_filename2, quiet=False)

print(f"File downloaded as: {output_filename2}")

Downloading...
From (original): https://drive.google.com/uc?id=1vAaazotBSAS6tAZ17l6d1x7rCjzRp5te
From (redirected): https://drive.google.com/uc?id=1vAaazotBSAS6tAZ17l6d1x7rCjzRp5te&confirm=t&uuid=af9e9a9d-a6d2-4696-975d-f0cdb758a6c8
To: /content/resume_dataset_with_features.csv
100%|██████████| 638M/638M [00:06<00:00, 105MB/s]

File downloaded as: resume_dataset_with_features.csv


In [6]:
import gdown

# Replace 'FILE_ID' with the actual file ID from your link
file_id3 = "1LW6366Q0CS2ZYvKiPrYezPGxbl7MCsnN"
output_filename3 = "hybrid_resume_parser.pkl"  # Change to desired output filename

# Construct the download URL
url = f"https://drive.google.com/uc?id={file_id3}"

# Download the file
gdown.download(url, output_filename3, quiet=False)

print(f"File downloaded as: {output_filename3}")

Downloading...
From (original): https://drive.google.com/uc?id=1LW6366Q0CS2ZYvKiPrYezPGxbl7MCsnN
From (redirected): https://drive.google.com/uc?id=1LW6366Q0CS2ZYvKiPrYezPGxbl7MCsnN&confirm=t&uuid=7a680567-7939-462d-86d3-1d030e97ff33
To: /content/hybrid_resume_parser.pkl
100%|██████████| 455M/455M [00:05<00:00, 78.4MB/s]

File downloaded as: hybrid_resume_parser.pkl


In [7]:
import pandas as pd
import numpy as np

In [9]:
clean_resume_data = pd.read_csv("/content/clean_resume_data.csv")
clean_resume_data.head()

,ID,Category,Feature
0,16852973,HR,hr administrator marketing associate hr admini...
1,22323967,HR,hr specialist hr operations summary media prof...
2,33176873,HR,hr director summary years experience recruitin...
3,27018550,HR,hr specialist summary dedicated driven dynamic...
4,17812897,HR,hr manager skill highlights hr skills hr depar...


In [ ]:
clean_resume_data.shape

(2484, 3)

In [ ]:
clean_resume_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2483 entries, 0 to 2483
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        2483 non-null   int64 
 1   Category  2483 non-null   object
 2   Feature   2483 non-null   object
dtypes: int64(1), object(2)
memory usage: 77.6+ KB


In [10]:
# drop row with missing values withou changine original dataframe
clean_resume_data.dropna(inplace=True)

In [ ]:
clean_resume_data['Category'].value_counts()

,count
Category,
INFORMATION-TECHNOLOGY,120
BUSINESS-DEVELOPMENT,119
FINANCE,118
ADVOCATE,118
ACCOUNTANT,118
ENGINEERING,118
CHEF,118
AVIATION,117
FITNESS,117


In [11]:
large_resume = pd.read_csv("/content/resume_dataset_with_features.csv")
large_resume.head()

,Role,Features
0,Social Media Manager,5 to 15 Years Digital Marketing Specialist M.T...
1,Frontend Web Developer,"2 to 12 Years Web Developer BCA HTML, CSS, Jav..."
2,Quality Control Manager,0 to 12 Years Operations Manager PhD Quality c...
3,Wireless Network Engineer,4 to 11 Years Network Engineer PhD Wireless ne...
4,Conference Manager,1 to 12 Years Event Manager MBA Event planning...


In [ ]:
large_resume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615940 entries, 0 to 1615939
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   Role      1615940 non-null  object
 1   Features  1615940 non-null  object
dtypes: object(2)
memory usage: 24.7+ MB


In [ ]:
large_resume.columns

Index(['Role', 'Features'], dtype='object')

In [ ]:
# printing all unique value counts of large_resume['Role'] column without skipping
print(large_resume['Role'].value_counts())
#large_resume['Role'].value_counts().sum()


Role
Interaction Designer            20580
Network Administrator           17470
User Interface Designer         14036
Social Media Manager            13945
User Experience Designer        13935
                                ...  
Inventory Control Specialist     3342
Budget Analyst                   3335
Clinical Nurse Manager           3324
Social Science Researcher        3321
Paid Advertising Specialist      3306
Name: count, Length: 376, dtype: int64


## Installing Dependancies

In [12]:
!pip install spacy flair transformers sentence-transformers scikit-learn joblib pandas pymupdf pdfplumber docx2txt streamlit
!python -m spacy download en_core_web_sm
!pip install tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 100.1 MB/s eta 0:00:00
 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 46.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Named Entity Recognition using Hybrid Approach: spaCy and BERT

Steps in the Hybrid Approach:
- Use spaCy for well-defined entity types (Name, Phone, Email, LinkedIn, Certifications).
- Use BERT (or Flair) for contextual entities (Job Role, Skills, Experience, Companies Worked For).
- Merge results to get a comprehensive resume parsing model.

## Hybrid Resume Parsing Code with Progress Bar

In [16]:
resume_df = pd.read_csv("/content/structured_resume_data.csv")
print(resume_df.columns)
resume_df.head()  # View the first few rows


Index(['Applicant Name', 'Job Role', 'Phone', 'Email', 'Companies Worked For',
       'Years of Work Experience', 'Skills', 'Referees', 'LinkedIn Profile',
       'Certifications', 'Education Background', 'Education Institutions'],
      dtype='object')


,Applicant Name,Job Role,Phone,Email,Companies Worked For,Years of Work Experience,Skills,Referees,LinkedIn Profile,Certifications,Education Background,Education Institutions
0,Darryl Rodriguez,"Manager, Analyst",+1-810-234-8247x16972,thompsonrichard@example.com,"State B, ##ANCI",12.0,"Excel, R",No,https://linkedin.com/in/ryan73,Certifications,"Master, Bachelor's","State B, ##ANCI"
1,Allison Joseph,"Technician, Engineering",476.373.3629x1110,cpacheco@example.com,"##CI, Current Engineering, HMA, ##P, ArcV, TOP...",3.0,"Improvement, Water Utility, Microsoft Access, ...",No,https://linkedin.com/in/murphybrian,Certified Data Analysis,M.Sc. Data Science,"##CI, Current Engineering, HMA, ##P, ArcV, TOP..."
2,Christopher Williams,Specialist,294.859.3140,diana99@example.org,Amazon,9.0,"Aid, Mass Index",No,https://linkedin.com/in/barnesmary,"Certifications, Training",Diploma,UC Berkeley
3,Elizabeth Cooper,-Manager,2008-2013,johnathan90@example.com,"Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec...",13.0,"NYSE Regulation, Amex, Floor, ##SE",No,https://linkedin.com/in/areed,"Certified, Certifications","Certificate, Certificate-Project, Certificate-LMC","Microsoft Office, K, Ex, NYSE, SF, NY, NY, Sec..."
4,Neil de Grasse Tyson,"Specialist, Manager/Supervisor",1997-2004,mcgrathtina@example.net,"MEDI, Chattanooga State, Tennessee Legislature...",15.0,"African American, Modern Times, Spanish, Spani...",No,https://linkedin.com/in/xmiller,Certified Cybersecurity,"Degrees, Master, Bachelor","MEDI, Chattanooga State, Tennessee Legislature..."


In [25]:
!pip install pdfminer.six

In [26]:
from pdfminer.high_level import extract_text
print("PDFMiner is successfully imported!")


PDFMiner is successfully imported!


In [32]:
import pandas as pd
import spacy
import re
import torch
from transformers import BertTokenizer, BertModel
from pdfminer.high_level import extract_text
import pickle


from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams


In [13]:
# Load CSV dataset
csv_file = "/content/resume_dataset_with_features.csv"
df = pd.read_csv(csv_file)

In [14]:
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Load BERT model & tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
!pip install transformers

In [33]:
'''
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)
'''
# Function to extract text from PDF using pdfminer.six
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        output_string = StringIO()
        resource_manager = PDFResourceManager()
        device = TextConverter(resource_manager, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(resource_manager, device)

        for page in PDFPage.get_pages(pdf_file):
            interpreter.process_page(page)

        text = output_string.getvalue()
        device.close()
        output_string.close()

    return text
# Define regex patterns
EMAIL_REGEX = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
PHONE_REGEX = r"(?:\+?\d{1,3}[ -]?)?(\(?\d{3}\)?[ -]?\d{3}[ -]?\d{4})"
LINKEDIN_REGEX = r"https?:\/\/(www\.)?linkedin\.com\/in\/[a-zA-Z0-9-_/]+"

# Function to extract named entities using spaCy
def extract_entities_spacy(text):
    doc = nlp(text)
    extracted_entities = {
        "Applicant Name": [],
        "Companies Worked For": [],
        "Certifications": [],
        "Education Background": [],
        "Education Institutions": []
    }

    for ent in doc.ents:
        if ent.label_ in ["PERSON"]:
            extracted_entities["Applicant Name"].append(ent.text)
        elif ent.label_ in ["ORG"]:
            extracted_entities["Companies Worked For"].append(ent.text)
        elif ent.label_ in ["EDUCATION"]:
            extracted_entities["Education Background"].append(ent.text)
        elif ent.label_ in ["GPE"]:
            extracted_entities["Education Institutions"].append(ent.text)

    return extracted_entities

# Function to extract skills using BERT embeddings
def extract_skills_bert(text, skills_list):
    tokenized_text = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = bert_model(**tokenized_text)

    extracted_skills = []
    for skill in skills_list:
        if re.search(skill.lower(), text.lower()):
            extracted_skills.append(skill)

    return extracted_skills

# Function to parse resume
def parse_resume(pdf_path, skills_list):
    resume_text = extract_text_from_pdf(pdf_path)

    parsed_data = {
        "Applicant Name": None,
        "Job Role": None,
        "Phone": None,
        "Email": None,
        "Companies Worked For": [],
        "Years of Work Experience": None,
        "Skills": [],
        "Referees": None,
        "LinkedIn Profile Link": None,
        "Certifications": [],
        "Education Background": [],
        "Education Institutions": []
    }

    # Extract entities
    spacy_entities = extract_entities_spacy(resume_text)
    parsed_data.update(spacy_entities)

    # Extract regex-based fields
    parsed_data["Email"] = re.findall(EMAIL_REGEX, resume_text)
    parsed_data["Phone"] = re.findall(PHONE_REGEX, resume_text)
    parsed_data["LinkedIn Profile Link"] = re.findall(LINKEDIN_REGEX, resume_text)

    # Extract skills using BERT
    parsed_data["Skills"] = extract_skills_bert(resume_text, skills_list)

    return parsed_data

# Save the hybrid model
model_data = {
    "nlp": nlp,
    "bert_model": bert_model,
    "tokenizer": tokenizer,
    "extract_text_from_pdf": extract_text_from_pdf,
    "extract_entities_spacy": extract_entities_spacy,
    "extract_skills_bert": extract_skills_bert,
    "parse_resume": parse_resume
}

with open("hybrid_resume_parser.pkl", "wb") as model_file:
    pickle.dump(model_data, model_file)



In [34]:
from pdfminer.high_level import extract_text

def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

if __name__ == '__main__':
    print(extract_text_from_pdf("/content/Moses Mugambi Data Analyst CV.pdf"))

Moses Mugambi 
Data Scientist 
Email: mugambimoses2@gmail.com | Phone: +254718695260 
LinkedIn:  linkedin.com/in/moses-mugambi-njeru  | GitHub: github.com/mk7890 

Professional Summary 

Motivated Data Scientist with expertise in Python, data analysis, visualizations, and machine 
learning. Proficient in SQL, Tableau, Excel, and web scraping. Experienced in predictive 
modelling, clustering, and NLP. Passionate about deriving insights from data to solve real-world 
problems. 

Work Experience 

•  Resume Parser. Built an NLP pipeline for extracting key resume details. 
•  Google Play Store Apps & YouTube Analysis: Data-driven insights using SQL, Pandas, 

and visualizations. 

•  Machine Learning Projects: Implemented Regression, Classification, Clustering, PCA, 

and CNNs. 

•  Time Series Analysis: Forecasted air quality trends using statistical models. 
•  NLP Sentiment Analysis: Analysed customer reviews for sentiment trends. 
•  Voice-Controlled Calculator: Developed a Python-base

In [35]:
# Test the resume parser with a sample file
test_pdf_path = "/content/Moses Mugambi Data Analyst CV.pdf"
skills_list = ["Python", "SQL", "Machine Learning", "Data Analysis", "Tableau"]
parsed_resume = parse_resume(test_pdf_path, skills_list)

# Print results in structured format
print("\nParsed Resume Data:")
for key, value in parsed_resume.items():
    print(f"{key}: {value}")


Parsed Resume Data:
Applicant Name: ['Scikit-Learn', 'Machine Learning']
Job Role: None
Phone: ['4718695260']
Email: ['mugambimoses2@gmail.com']
Companies Worked For: ['Mugambi', 'Data Scientist', 'GitHub', 'NLP', 'NLP', 'SQL', 'Pandas', '•  Machine Learning Projects: Implemented Regression, Classification, Clustering', 'PCA', '•  NLP Sentiment Analysis', '•  Voice-Controlled Calculator', 'NLP', 'Data Visualization', 'Analytical Thinking', 'The Technical University', 'Moi High School Mbiruri']
Years of Work Experience: None
Skills: ['Python', 'SQL', 'Machine Learning', 'Data Analysis', 'Tableau']
Referees: None
LinkedIn Profile Link: []
Certifications: []
Education Background: []
Education Institutions: ['LinkedIn', 'Python', 'SQL', 'Tableau', 'SQL', 'Tableau', 'Web Scraping']


In [36]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [37]:
# Your public ip is the password to the localtunnel
!curl ipv4.icanhazip.com

34.106.190.96


In [29]:
!pip install streamlit

In [39]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦your url is: https://nasty-monkeys-trade.loca.lt
^C
